In [1]:
import sys
import os
import pandas as pd
import numpy as np
from Bio import SeqIO
import re

In [2]:
dataset = 'JVO2010'

print('Loading raw data for', dataset, '...')
data = pd.read_csv('/Users/maryamkoddus/Documents/maryam-ko-QMUL-MSc-Project/01_input_data/raw_data/supplementarytable_s2.csv', header=0)
print('Raw data loaded.')
data

Loading raw data for JVO2010 ...
Raw data loaded.


/var/folders/1v/y_z33m9d25d2tszlfrvnp2300000gn/T/ipykernel_3527/862985499.py:4: DtypeWarning: Columns (2,18,58) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('/Users/maryamkoddus/Documents/maryam-ko-QMUL-MSc-Project/01_input_data/raw_data/supplementarytable_s2.csv', header=0)


,Protein ID (IPI v.3.37 number),Uniprot ID (Swiss-Prot or TrEMBL),Phosphosite position in protein,Known Site in Expasy (Swiss-Prot or TrEMBL),Phosphosite Localized?,Ratio M/L Normalized by Protein Noco45 - Log2 mitosis,Ratio M/L Normalized by Protein Noco3h - Log2 G1,Ratio M/L Normalized by Protein ThymON - Log2 G1/S,Ratio H/M Normalized by Protein Noco45 - Log2 early S,Ratio H/M Normalized by Protein Noco3h - Log2 Late S,...,PKA motif?,AKT motif?,PKC motif?,CAMK2 motif?,DNA damage?,DNA repair?,DNA replication?,Cell-cycle checkpoint?,signal transduction?,apoptosis?
0,IPI00000006,P01112;P78460;Q7KZ06;Q92468;Q9BR65;Q9UM97;Q9UCE2,145,NaN,YES (Class1),0.00,0.00,0.00,0.00,0.00,...,NO,NO,NO,NO,NO,NO,NO,NO,YES,YES
1,IPI00000015,Q08170;A8K644;Q53F45;Q59EF5,78,NaN,YES (Class1),0.96,0.00,0.00,-1.97,0.00,...,NO,NO,NO,YES,NO,NO,NO,NO,NO,NO
2,IPI00000015,Q08170;A8K644;Q53F45;Q59EF5,288,NaN,YES (Class1),-0.52,-0.32,0.00,-0.42,-0.76,...,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO
3,IPI00000015,Q08170;A8K644;Q53F45;Q59EF5,290,NaN,YES (Class1),-0.52,-0.32,0.00,-0.42,-0.76,...,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO
4,IPI00000015,Q08170;A8K644;Q53F45;Q59EF5,292,NaN,YES (Class1),-0.52,-0.32,0.00,-0.42,-0.76,...,NO,NO,NO,YES,NO,NO,NO,NO,NO,NO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24709,IPI00889161,NaN,268,NaN,No (Class3),3.56,0.00,0.00,0.00,0.00,...,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO
24710,IPI00892604,A2BHY4;Q6U2F8;Q6U2G0;Q6U2G1;A6H8M8;A7E2V2;B0QZ...,900,NaN,No (Class3),0.00,0.00,0.37,0.00,0.00,...,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO
24711,IPI00893197,A9IZU4;Q15257-1;Q15257;Q5T948;A6PVN5;Q15257-2;...,141,NaN,No (Class3),NaN,NaN,NaN,NaN,NaN,...,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO
24712,IPI00893844,Q7Z6J0-1;Q7Z6J0;Q7Z6J0-2;Q7Z6J0-3,54,NaN,No (Class3),NaN,NaN,NaN,NaN,NaN,...,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO


In [3]:
# filter data to keep only those with localization probability >= 0.85
data = data[data['P-site Loc. Prob.'] >= 0.85] 

# Filtering out semi-colons from 'Amino acid', 'Positions within proteins', and 'Gene names' columns
data = data[~data['Amino Acid'].str.contains(';', na=False)]
data = data[~data['Phosphosite position in protein'].str.contains(';', na=False)]
data = data[~data['Gene Names'].str.contains(';', na=False)]
data

,Protein ID (IPI v.3.37 number),Uniprot ID (Swiss-Prot or TrEMBL),Phosphosite position in protein,Known Site in Expasy (Swiss-Prot or TrEMBL),Phosphosite Localized?,Ratio M/L Normalized by Protein Noco45 - Log2 mitosis,Ratio M/L Normalized by Protein Noco3h - Log2 G1,Ratio M/L Normalized by Protein ThymON - Log2 G1/S,Ratio H/M Normalized by Protein Noco45 - Log2 early S,Ratio H/M Normalized by Protein Noco3h - Log2 Late S,...,PKA motif?,AKT motif?,PKC motif?,CAMK2 motif?,DNA damage?,DNA repair?,DNA replication?,Cell-cycle checkpoint?,signal transduction?,apoptosis?
11,IPI00000030,Q14738-1;Q14738;A8K6Y2;Q59EF0,88,NaN,YES (Class1),2.72,1.35,0.00,0.07,0.87,...,YES,YES,NO,YES,NO,NO,NO,NO,YES,NO
12,IPI00000030,Q14738-1;Q14738;A8K6Y2;Q59EF0,89,NaN,YES (Class1),2.72,2.11,1.78,0.07,0.87,...,NO,NO,YES,YES,NO,NO,NO,NO,YES,NO
13,IPI00000030,Q14738-1;Q14738;A8K6Y2;Q59EF0,90,NaN,YES (Class1),2.72,1.96,1.78,0.07,0.87,...,NO,NO,NO,NO,NO,NO,NO,NO,YES,NO
16,IPI00000070,P01130;Q59FQ1;Q6LCH2;Q6RCM4;Q9UH51,397,NaN,YES (Class1),0.00,0.00,0.00,0.00,0.00,...,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO
17,IPI00000070,P01130;Q59FQ1;Q6LCH2;Q6RCM4;Q9UH51,404,NaN,YES (Class1),0.00,0.00,0.00,0.00,0.00,...,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20430,IPI00896427,Q5T0Z8-1;Q5T0Z8,666,NaN,YES (Class1),NaN,NaN,NaN,NaN,NaN,...,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO
20431,IPI00896427,Q5T0Z8-1;Q5T0Z8,837,NaN,YES (Class1),NaN,NaN,NaN,NaN,NaN,...,YES,NO,NO,NO,NO,NO,NO,NO,NO,NO
20432,IPI00896430,B2RXJ5,873,NaN,YES (Class1),NaN,NaN,NaN,NaN,NaN,...,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO
20439,IPI00896449,B2RUZ4,22,NaN,YES (Class1),-1.67,0.00,0.00,0.00,0.00,...,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO


In [4]:
# filter data
data['Sequence Window'] = data['Sequence Window'].str.replace('_', '')
data

,Protein ID (IPI v.3.37 number),Uniprot ID (Swiss-Prot or TrEMBL),Phosphosite position in protein,Known Site in Expasy (Swiss-Prot or TrEMBL),Phosphosite Localized?,Ratio M/L Normalized by Protein Noco45 - Log2 mitosis,Ratio M/L Normalized by Protein Noco3h - Log2 G1,Ratio M/L Normalized by Protein ThymON - Log2 G1/S,Ratio H/M Normalized by Protein Noco45 - Log2 early S,Ratio H/M Normalized by Protein Noco3h - Log2 Late S,...,PKA motif?,AKT motif?,PKC motif?,CAMK2 motif?,DNA damage?,DNA repair?,DNA replication?,Cell-cycle checkpoint?,signal transduction?,apoptosis?
11,IPI00000030,Q14738-1;Q14738;A8K6Y2;Q59EF0,88,NaN,YES (Class1),2.72,1.35,0.00,0.07,0.87,...,YES,YES,NO,YES,NO,NO,NO,NO,YES,NO
12,IPI00000030,Q14738-1;Q14738;A8K6Y2;Q59EF0,89,NaN,YES (Class1),2.72,2.11,1.78,0.07,0.87,...,NO,NO,YES,YES,NO,NO,NO,NO,YES,NO
13,IPI00000030,Q14738-1;Q14738;A8K6Y2;Q59EF0,90,NaN,YES (Class1),2.72,1.96,1.78,0.07,0.87,...,NO,NO,NO,NO,NO,NO,NO,NO,YES,NO
16,IPI00000070,P01130;Q59FQ1;Q6LCH2;Q6RCM4;Q9UH51,397,NaN,YES (Class1),0.00,0.00,0.00,0.00,0.00,...,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO
17,IPI00000070,P01130;Q59FQ1;Q6LCH2;Q6RCM4;Q9UH51,404,NaN,YES (Class1),0.00,0.00,0.00,0.00,0.00,...,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20430,IPI00896427,Q5T0Z8-1;Q5T0Z8,666,NaN,YES (Class1),NaN,NaN,NaN,NaN,NaN,...,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO
20431,IPI00896427,Q5T0Z8-1;Q5T0Z8,837,NaN,YES (Class1),NaN,NaN,NaN,NaN,NaN,...,YES,NO,NO,NO,NO,NO,NO,NO,NO,NO
20432,IPI00896430,B2RXJ5,873,NaN,YES (Class1),NaN,NaN,NaN,NaN,NaN,...,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO
20439,IPI00896449,B2RUZ4,22,NaN,YES (Class1),-1.67,0.00,0.00,0.00,0.00,...,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO


In [5]:
def match_seq_to_genename(dataset, seq_column):
    '''
    Maps amino acid sequences to gene names using the loaded fasta file.
    
    args:
    =====
    dataset: <pd.Dataframe> with a column of amino acid sequences
    seq_column: <str> column name containing amino acid sequences
    
    out:
    ====
    dataset: <pd.Dataframe> with an additional column containing gene names
    '''    

    fasta_sequence = list(SeqIO.parse(open(f'/Users/maryamkoddus/Documents/maryam-ko-QMUL-MSc-Project/01_input_data/raw_data/UP000005640_9606.fasta'), "fasta"))
    
    
    gene_dict = {}
    
    # iterate over rows in seq_column
    for i in dataset[seq_column]:
        print(i)
        i_str = str(i)
        for seq_record in fasta_sequence:
            matches = re.findall(i_str, str(seq_record.seq))
            if matches:
                print(f"Match found for sequence: {seq_record}")
                gene_name_match = re.search(r"GN=(\w+)", seq_record.description)
                print('Gene name match:', gene_name_match)
                # gene_name_match = re.search("GN=(\w+)", seq_record.description)
                if gene_name_match:
                    gene_name = gene_name_match.group(1)
                    gene_dict[i] = gene_name
                    print(f"Match found: {i_str} -> {gene_name}")
                else: 
                    print(f"No gene name found in description for sequence: {i_str}")
    
    # map sequences to gene names           
    dataset['GeneName'] = dataset[seq_column].map(gene_dict) 
    print('Amino acid sequences matched to gene names.')
    return dataset 

In [6]:
data = match_seq_to_genename(data, 'Sequence Window')

KKERRQSSSRFNL
Match found for sequence: ID: sp|Q14738|2A5D_HUMAN
Name: sp|Q14738|2A5D_HUMAN
Description: sp|Q14738|2A5D_HUMAN Serine/threonine-protein phosphatase 2A 56 kDa regulatory subunit delta isoform OS=Homo sapiens OX=9606 GN=PPP2R5D PE=1 SV=1
Number of features: 0
Seq('MPYKLKKEKEPPKVAKCTAKPSSSGKDGGGENTEEAQPQPQPQPQPQAQSQPPS...EAL')
Gene name match: <re.Match object; span=(125, 135), match='GN=PPP2R5D'>
Match found: KKERRQSSSRFNL -> PPP2R5D
KERRQSSSRFNLS
Match found for sequence: ID: sp|Q14738|2A5D_HUMAN
Name: sp|Q14738|2A5D_HUMAN
Description: sp|Q14738|2A5D_HUMAN Serine/threonine-protein phosphatase 2A 56 kDa regulatory subunit delta isoform OS=Homo sapiens OX=9606 GN=PPP2R5D PE=1 SV=1
Number of features: 0
Seq('MPYKLKKEKEPPKVAKCTAKPSSSGKDGGGENTEEAQPQPQPQPQPQAQSQPPS...EAL')
Gene name match: <re.Match object; span=(125, 135), match='GN=PPP2R5D'>
Match found: KERRQSSSRFNLS -> PPP2R5D
ERRQSSSRFNLSK
Match found for sequence: ID: sp|Q14738|2A5D_HUMAN
Name: sp|Q14738|2A5D_HUMAN
Descri

In [7]:
data['Phosphosite'] = data['Amino Acid'].astype(str) + '(' + data['Phosphosite position in protein'].astype(str) + ')'


In [8]:
# Keep only 'Phosphosite' and ratio columns
keepcols = ['Phosphosite'] + ['GeneName'] + [col for col in data.columns if 'Ratio' in col]
data = data[keepcols]
data

,Phosphosite,GeneName,Ratio M/L Normalized by Protein Noco45 - Log2 mitosis,Ratio M/L Normalized by Protein Noco3h - Log2 G1,Ratio M/L Normalized by Protein ThymON - Log2 G1/S,Ratio H/M Normalized by Protein Noco45 - Log2 early S,Ratio H/M Normalized by Protein Noco3h - Log2 Late S,Ratio H/M Normalized by Protein ThymON - Log2 G2,Ratio Variability [%],Log2 Ratio M/L Normalized aphidicolin,Log2 Ratio H/M Normalized aphidicolin
11,S(88),PPP2R5D,2.72,1.35,0.00,0.07,0.87,0.00,11.16,0.0,0.0
12,S(89),PPP2R5D,2.72,2.11,1.78,0.07,0.87,1.92,8.73,0.0,0.0
13,S(90),PPP2R5D,2.72,1.96,1.78,0.07,0.87,1.92,10.59,0.0,0.0
16,S(397),LDLR,0.00,0.00,0.00,0.00,0.00,0.00,n.a.,0.0,0.0
17,T(404),LDLR,0.00,0.00,0.00,0.00,0.00,0.00,n.a.,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
20430,T(666),C6orf132,NaN,NaN,NaN,NaN,NaN,NaN,n.a.,NaN,NaN
20431,S(837),C6orf132,NaN,NaN,NaN,NaN,NaN,NaN,n.a.,NaN,NaN
20432,S(873),NaN,NaN,NaN,NaN,NaN,NaN,NaN,n.a.,NaN,NaN
20439,S(22),SMIM1,-1.67,0.00,0.00,0.00,0.00,0.00,n.a.,0.0,0.0


In [9]:
Ratio_columns = [col for col in data.columns if 'Ratio' in col]
data[Ratio_columns] = data[Ratio_columns].apply(pd.to_numeric, errors='coerce')

/var/folders/1v/y_z33m9d25d2tszlfrvnp2300000gn/T/ipykernel_3527/881820786.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[Ratio_columns] = data[Ratio_columns].apply(pd.to_numeric, errors='coerce')


In [10]:
def create_phos_ID(dataset):
    '''
    Concatenates GeneName and Phosphosite columns.
    
    args:
    =====
    dataset: <pd.Dataframe> with columns 'GeneName' and 'Phosphosite'
    
    out:
    ====
    dataset: <pd.Dataframe> with 'phosphosite_ID' column and 'GeneName' + 'Phosphosite' columns dropped
    '''
    dataset.loc[:, 'phosphosite_ID'] = dataset['GeneName'].astype(str) + '_' + dataset['Phosphosite'].astype(str)
    dataset = dataset.drop(columns=['Phosphosite', 'GeneName'])
    print('Phosphosite IDs created.')
    return dataset

data = create_phos_ID(data) # call function to create phosphosite_ID column

print('Phosphosite IDs created.')

Phosphosite IDs created.
Phosphosite IDs created.


/var/folders/1v/y_z33m9d25d2tszlfrvnp2300000gn/T/ipykernel_3527/3553972964.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset.loc[:, 'phosphosite_ID'] = dataset['GeneName'].astype(str) + '_' + dataset['Phosphosite'].astype(str)


In [11]:
def clean_phosID_col(data):
    data = data[~data.phosphosite_ID.str.contains('nan', case = False)]
    data = data[~data.phosphosite_ID.str.contains(';', case = False)] # remove rows containing ';' in phosphosite_ID column
    data = data[~data.phosphosite_ID.str.contains('-', case = False)] # remove rows containing '-' in phosphosite_ID column
    
    # check whether there are any phosphosites with multiple measurements
    data_grouped = data.groupby(by = 'phosphosite_ID')
    if len(data) != len(data_grouped):
        data = data_grouped.mean()
        data.reset_index(inplace=True) # reset index
        print('Phosphosites with multiple measurements have been averaged')
    else:
        print('There are no phosphosites with multiple measurements')
        
    print(data)
        
    data = data.replace([np.inf, -np.inf], np.nan)
        
    if data.columns[0] != 'phosphosite_ID':
        phosphosite_ID = data.pop('phosphosite_ID')
        data.insert(0, 'phosphosite_ID', phosphosite_ID)
    return data

In [12]:
data = clean_phosID_col(data)
print("After cleaning phosphosite_ID column:")
data

Phosphosites with multiple measurements have been averaged
     phosphosite_ID  Ratio M/L Normalized by Protein Noco45 - Log2 mitosis  \
0      AAGAB_S(310)                                              -0.27       
1      AAGAB_S(311)                                              -0.27       
2      AARS1_S(399)                                               2.25       
3      AARS1_S(555)                                              -0.96       
4      AARS1_T(941)                                               0.00       
...             ...                                                ...       
2087   ZNHIT6_S(11)                                               1.71       
2088  ZSWIM9_S(277)                                               0.00       
2089  ZSWIM9_S(281)                                               1.41       
2090    ZW10_T(133)                                               0.00       
2091    ZW10_T(438)                                               0.00       

    

,phosphosite_ID,Ratio M/L Normalized by Protein Noco45 - Log2 mitosis,Ratio M/L Normalized by Protein Noco3h - Log2 G1,Ratio M/L Normalized by Protein ThymON - Log2 G1/S,Ratio H/M Normalized by Protein Noco45 - Log2 early S,Ratio H/M Normalized by Protein Noco3h - Log2 Late S,Ratio H/M Normalized by Protein ThymON - Log2 G2,Ratio Variability [%],Log2 Ratio M/L Normalized aphidicolin,Log2 Ratio H/M Normalized aphidicolin
0,AAGAB_S(310),-0.27,0.04,0.14,-0.08,0.29,-0.13,5.25,0.0,0.0
1,AAGAB_S(311),-0.27,0.04,0.14,-0.08,0.29,-0.13,5.25,0.0,0.0
2,AARS1_S(399),2.25,2.27,0.00,0.28,-0.67,0.00,5.16,0.0,0.0
3,AARS1_S(555),-0.96,0.12,0.00,1.08,0.39,0.00,NaN,0.0,0.0
4,AARS1_T(941),0.00,0.00,0.00,0.00,0.00,0.00,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
2087,ZNHIT6_S(11),1.71,0.00,0.00,-0.06,0.00,0.00,NaN,NaN,NaN
2088,ZSWIM9_S(277),0.00,0.00,1.37,0.00,0.00,0.57,NaN,0.0,0.0
2089,ZSWIM9_S(281),1.41,1.38,1.55,-0.01,0.32,1.06,NaN,0.0,0.0
2090,ZW10_T(133),0.00,0.00,0.00,0.00,0.00,0.00,NaN,NaN,NaN


In [13]:
data.to_csv(f'/Users/maryamkoddus/Documents/maryam-ko-QMUL-MSc-Project/01_input_data/PreprocessedDatasets/JVO2010.csv', index=False)


print(dataset, 'has been saved to CSV successfully!', data)

JVO2010 has been saved to CSV successfully!      phosphosite_ID  Ratio M/L Normalized by Protein Noco45 - Log2 mitosis  \
0      AAGAB_S(310)                                              -0.27       
1      AAGAB_S(311)                                              -0.27       
2      AARS1_S(399)                                               2.25       
3      AARS1_S(555)                                              -0.96       
4      AARS1_T(941)                                               0.00       
...             ...                                                ...       
2087   ZNHIT6_S(11)                                               1.71       
2088  ZSWIM9_S(277)                                               0.00       
2089  ZSWIM9_S(281)                                               1.41       
2090    ZW10_T(133)                                               0.00       
2091    ZW10_T(438)                                               0.00       

      Ratio M/L Nor